# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

### Metrics to consider:
- Monthly active users
- Daily active users in past month
- Total paid and unpaid users
- Total ads served in the past month
- Cohort per Month - % of users cancelled, % of users upgrades


In [24]:
pip install --upgrade pyspark


     |████████████████████████████████| 212.3 MB 14 kB/s  eta 0:00:01    |█▋                              | 10.7 MB 870 kB/s eta 0:03:52     |███████████████████▏            | 127.2 MB 1.0 MB/s eta 0:01:22     |███████████████████▋            | 130.2 MB 1.3 MB/s eta 0:01:05
     |████████████████████████████████| 198 kB 1.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6d6988207ecab299522cb7140689bd31a429e1bddde5eff6311978db4dce0ed4
  Stored in directory: /home/freemo/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.7
    Uninstalling py4j-0.10.7:
      Successfully uninstalled py4j-0.10.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 2.4.0
    Uninstalling pyspark-2.4.0:
      Successfully uninstalled pyspark-2.4.0
Note: you may need to restart the kernel to use updat

In [23]:
# import libraries
import seaborn as sns
import pandas as pd
import numpy as np
import datetime # missing and together with "pip install --upgrade pyspark" were causing issues with time series
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from plotly.offline import iplot
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
import cufflinks as cf
cf.go_offline()
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [2]:
# create a Spark session
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
spark

In [4]:
print(spark.catalog.listTables())

[]


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
pwd

'/home/freemo/Projects/SparkProject'

In [6]:
mini = '/home/freemo/Projects/largeData/mini_sparkify_event_data.json'
medium = '/home/freemo/Projects/largeData/medium_sparkify_event_data.json'

In [7]:
df = spark.read.json(mini)

In [8]:
df.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

In [9]:
df.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [10]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [11]:
# check columns with Null values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [12]:
# missing values in userID
df.select([count(when(isnan('userID'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(userID) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [13]:
# missing values in sessionID
df.select([count(when(isnan('sessionID'),True))]).show()

+-----------------------------------------------+
|count(CASE WHEN isnan(sessionID) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+



In [24]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

In [25]:
df = df.withColumn("hour", get_hour(df.ts))

In [26]:
songs_in_hour = df.filter(df.page == "NextSong").groupby(df.hour).count().orderBy(df.hour.cast("float"))

In [27]:
songs_in_hour.show()

+----+-----+
|hour|count|
+----+-----+
|   0|10262|
|   1| 9908|
|   2| 9327|
|   3| 8895|
|   4| 8339|
|   5| 8359|
|   6| 8229|
|   7| 7979|
|   8| 8114|
|   9| 8119|
|  10| 8225|
|  11| 8359|
|  12| 8602|
|  13| 8653|
|  14| 9225|
|  15| 9479|
|  16|10047|
|  17|11036|
|  18|11381|
|  19|11568|
+----+-----+
only showing top 20 rows



In [29]:
songs_in_hour_pd = songs_in_hour.toPandas()
songs_in_hour_pd.hour = pd.to_numeric(songs_in_hour_pd.hour)

In [35]:
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
day_ind = [1,2,3,4,5,6,7]
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).hour, IntegerType() )
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).day, IntegerType() )
get_wkday = udf(lambda x: day_name[datetime.datetime.fromtimestamp(x/1000).weekday()] )

get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

In [36]:
df = df.withColumn('ts_hour', get_hour('ts'))
df = df.withColumn('ts_day', get_day('ts'))
df = df.withColumn('ts_wkday', get_wkday('ts'))

In [42]:
df.select('ts_hour').show(7)

+-------+
|ts_hour|
+-------+
|      3|
|      3|
|      3|
|      3|
|      3|
|      3|
|      3|
+-------+
only showing top 7 rows



In [44]:
df.select('ts_day').show(7)

+------+
|ts_day|
+------+
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
+------+
only showing top 7 rows



In [45]:
df.select('ts_wkday').show(7)

+--------+
|ts_wkday|
+--------+
|  Monday|
|  Monday|
|  Monday|
|  Monday|
|  Monday|
|  Monday|
|  Monday|
+--------+
only showing top 7 rows



In [55]:
df.groupby('ts_wkday').count().show()

+---------+-----+
| ts_wkday|count|
+---------+-----+
|Wednesday|48927|
|  Tuesday|47827|
|   Friday|49537|
| Thursday|45147|
| Saturday|31137|
|   Monday|41785|
|   Sunday|22140|
+---------+-----+



In [56]:
df.groupby('ts_hour').count().show()

+-------+-----+
|ts_hour|count|
+-------+-----+
|      7|10098|
|     15|12025|
|     11|10327|
|      3|11086|
|      8|10116|
|     22|13633|
|     16|12699|
|      0|12853|
|      5|10392|
|     18|14376|
|     17|13946|
|      6|10318|
|     19|14731|
|     23|13566|
|      9|10160|
|      1|12334|
|     20|14219|
|     10|10254|
|      4|10462|
|     12|10725|
+-------+-----+
only showing top 20 rows



In [58]:
df.groupby('ts_day').count().show(31)

+------+-----+
|ts_day|count|
+------+-----+
|    31| 5359|
|    28| 7472|
|    26| 9920|
|    27|10096|
|    12| 9092|
|    22| 8235|
|     1|11107|
|    13| 8208|
|     6| 8773|
|    16|11475|
|     3| 9097|
|    20|11559|
|     5| 8572|
|    19| 9878|
|    15|10552|
|     9|11628|
|    17| 8350|
|     4| 7136|
|     8| 8734|
|    23| 9894|
|     7| 7596|
|    10| 9716|
|    25| 8375|
|    24| 8722|
|    29| 9639|
|    21| 9826|
|    11| 9165|
|    14| 7427|
|     2|11989|
|    30|11241|
|    18| 7667|
+------+-----+



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.